In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Twente/Q2/RPCN---November-2025

/content/drive/MyDrive/Twente/Q2/RPCN---November-2025


In [7]:
#!/usr/bin/env python3
import os
import pandas as pd
import numpy as np

def calibrate_imu_series(directory: str, g: float = 9.80665):
    """
    Calibrate an IMU from 18 CSV files (space-separated, headers as given).

    Files must be named like  MT_001-000.csv , MT_002-000.csv , ... MT_018-000.csv
    (the numeric part is used for ordering).

    Returns a dict with per-series and overall results.
    """
    # ------------------------------------------------------------
    # 1. List & sort files
    # ------------------------------------------------------------
    csv_files = [f for f in os.listdir(directory) if f.lower().endswith('.csv')]
    if len(csv_files) != 18:
        raise ValueError(f"Expected 18 CSV files, found {len(csv_files)} in {directory}")

    # sort by the number in the filename (001 … 018)
    csv_files.sort(key=lambda x: int(x.split('_')[-1].split('-')[0]))

    # ------------------------------------------------------------
    # 2. Helper: average of a column in one file
    # ------------------------------------------------------------
    def avg_column(filepath: str, col: str) -> float:
        df = pd.read_csv(filepath, sep=r'\s+', usecols=[col])
        return float(df[col].mean())

    # ------------------------------------------------------------
    # 3. Define which files belong to which orientation
    # ------------------------------------------------------------
    # 0-based indices for the three series (0-5, 6-11, 12-17)
    series_idxs = [(0, 6, 12), (1, 7, 13), (2, 8, 14), (3, 9, 15), (4, 10, 16), (5, 11, 17)]

    # Axis → (up indices, down indices, column name)
    axis_cfg = {
        'Z': ( (1, 7, 13), (0, 6, 12), 'Acc_Z' ),   # 001-down, 002-up
        'X': ( (2, 8, 14), (3, 9, 15), 'Acc_Y' ),   # 003-down, 004-up
        'Y': ( (4, 10, 16), (5, 11, 17), 'Acc_X' ) # 006-down, 005-up
    }

    # ------------------------------------------------------------
    # 4. Compute per-series values
    # ------------------------------------------------------------
    results = {'Z': [], 'X': [], 'Y': []}

    for axis, (up_idx, down_idx, col) in axis_cfg.items():
        for ser in range(3):                     # three series
            # files for this series, this orientation
            up_files   = [os.path.join(directory, csv_files[i]) for i in up_idx[ser:ser+1]]
            down_files = [os.path.join(directory, csv_files[i]) for i in down_idx[ser:ser+1]]

            f_up   = avg_column(up_files[0], col)
            f_down = avg_column(down_files[0], col)

            b = (f_up + f_down) / 2.0
            S = (f_up - f_down - 2 * g) / (2 * g)

            results[axis].append({'series': ser + 1, 'f_up': f_up, 'f_down': f_down,
                                  'b': b, 'S': S})

    # ------------------------------------------------------------
    # 5. Overall average
    # ------------------------------------------------------------
    overall = {}
    for axis in results:
        b_vals = [r['b'] for r in results[axis]]
        S_vals = [r['S'] for r in results[axis]]
        overall[axis] = {'b_avg': np.mean(b_vals), 'S_avg': np.mean(S_vals)}

    # ------------------------------------------------------------
    # 6. Pretty printing
    # ------------------------------------------------------------
    def print_series(axis: str):
        print(f"\n=== {axis}-axis calibration ===")
        print(f"{'Series':>6}  {'f_up':>10}  {'f_down':>10}  {'b':>12}  {'S':>12}")
        print("-" * 52)
        for r in results[axis]:
            print(f"{r['series']:6d}  {r['f_up']:10.6f}  {r['f_down']:10.6f}  "
                  f"{r['b']:12.6f}  {r['S']:12.6f}")
        print("-" * 52)
        print(f"{'AVG':>6}  {'':>10}  {'':>10}  {overall[axis]['b_avg']:12.6f}  {overall[axis]['S_avg']:12.6f}")

    print_series('Z')
    print_series('X')
    print_series('Y')

    # ------------------------------------------------------------
    # 7. Return structured data (optional)
    # ------------------------------------------------------------
    return {'per_series': results, 'overall': overall}


calibrate_imu_series('/content/drive/MyDrive/Twente/Q2/RPCN---November-2025/Assignment 1/Data/csv')



=== Z-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
     1   -9.818758    9.806066     -0.006346     -2.000588
     2   -9.816788    9.806664     -0.005062     -2.000518
     3   -9.822028    9.803234     -0.009397     -2.000610
----------------------------------------------------
   AVG                             -0.006935     -2.000572

=== X-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
     1   -9.745906    9.868271      0.061183     -2.000045
     2   -9.747658    9.869006      0.060674     -2.000172
     3   -9.747367    9.867701      0.060167     -2.000090
----------------------------------------------------
   AVG                              0.060675     -2.000102

=== Y-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
  

{'per_series': {'Z': [{'series': 1,
    'f_up': -9.818758250835423,
    'f_down': 9.80606623488416,
    'b': -0.006346007975631451,
    'S': -2.0005875852467248},
   {'series': 2,
    'f_up': -9.816787849124381,
    'f_down': 9.806663514657263,
    'b': -0.005062167233559123,
    'S': -2.000517575511599},
   {'series': 3,
    'f_up': -9.822028428029546,
    'f_down': 9.80323360773231,
    'b': -0.009397410148618057,
    'S': -2.0006098940903296}],
  'X': [{'series': 1,
    'f_up': -9.745906166734777,
    'f_down': 9.868271201150565,
    'b': 0.061182517207893916,
    'S': -2.0000447333128717},
   {'series': 2,
    'f_up': -9.747658021970555,
    'f_down': 9.86900603359056,
    'b': 0.06067400581000282,
    'S': -2.0001715190998515},
   {'series': 3,
    'f_up': -9.74736670509735,
    'f_down': 9.867701250223615,
    'b': 0.060167272563131924,
    'S': -2.0000901406352303}],
  'Y': [{'series': 1,
    'f_up': -9.850110873937957,
    'f_down': 9.779993890153083,
    'b': -0.03505849189243

In [8]:
#!/usr/bin/env python3
import os
import pandas as pd
import numpy as np

# Earth rotation rate (rad/s) at latitude ≈ 0°–52° (standard value)
OMEGA_E = 7.292115e-5  # rad/s

def calibrate_gyro_earth_rotation(directory: str, g_nominal: float = 9.80665):
    """
    Calibrate gyroscope bias (b_g) and scale error (S_g) using Earth's rotation.
    Uses the same 18 CSV files as accelerometer calibration.
    """
    # ------------------------------------------------------------
    # 1. List & sort files (001 ... 018)
    # ------------------------------------------------------------
    csv_files = [f for f in os.listdir(directory) if f.lower().endswith('.csv')]
    if len(csv_files) != 18:
        raise ValueError(f"Expected 18 CSV files, found {len(csv_files)}")

    csv_files.sort(key=lambda x: int(x.split('_')[-1].split('-')[0]))
    filepaths = [os.path.join(directory, f) for f in csv_files]

    # ------------------------------------------------------------
    # 2. Helper: column average
    # ------------------------------------------------------------
    def avg_col(fp, col):
        return float(pd.read_csv(fp, sep=r'\s+', usecols=[col])[col].mean())

    # ------------------------------------------------------------
    # 3. Determine nominal Z-axis and compute sin(phi) for each file
    # ------------------------------------------------------------
    sin_phi_per_file = []
    nominal_z_axis_per_file = []  # 'X', 'Y', or 'Z'
    omega_z_per_file = []
    omega_xy_per_file = []

    for fp in filepaths:
        ax = avg_col(fp, 'Acc_X')
        ay = avg_col(fp, 'Acc_Y')
        az = avg_col(fp, 'Acc_Z')

        # Find which sensor axis has largest |acceleration| → that's nominal Z
        abs_vals = np.abs([ax, ay, az])
        idx = int(np.argmax(abs_vals))
        axis_map = ['X', 'Y', 'Z']
        nominal_z = axis_map[idx]

        # Measured gravity vector in sensor frame
        g_vec = np.array([ax, ay, az])
        g_sensor = g_vec[idx]  # signed value along nominal Z

        # Gyro readings
        gx = avg_col(fp, 'Gyr_X')
        gy = avg_col(fp, 'Gyr_Y')
        gz = avg_col(fp, 'Gyr_Z')
        g_all = np.array([gx, gy, gz])

        # Component along nominal Z
        omega_z = g_all[idx]

        # Horizontal components (perpendicular to nominal Z)
        omega_horizontal = np.delete(g_all, idx)  # remove nominal Z component
        omega_xy = np.linalg.norm(omega_horizontal)

        # sin(phi) = omega_xy / omega_e
        # (phi = co-latitude, sin(phi) = |omega_horizontal| / omega_e)
        sin_phi = omega_xy / OMEGA_E if OMEGA_E > 0 else 0.0

        sin_phi_per_file.append(sin_phi)
        nominal_z_axis_per_file.append(nominal_z)
        omega_z_per_file.append(omega_z)
        omega_xy_per_file.append(omega_xy)

    # ------------------------------------------------------------
    # 4. Pair up/down files (same as accelerometer)
    # ------------------------------------------------------------
    pairs = [
        (0, 1), (6, 7), (12, 13),   # Z-axis up/down pairs
        (2, 3), (8, 9), (14, 15),   # X-axis up/down pairs
        (4, 5), (10, 11), (16, 17)  # Y-axis up/down pairs
    ]

    results = {'X': [], 'Y': [], 'Z': []}

    for axis in ['X', 'Y', 'Z']:
        # Determine which file index corresponds to this sensor axis being nominal Z
        axis_indices = []
        if axis == 'X':
            axis_indices = [2, 3, 8, 9, 14, 15]   # files where X is nominal Z
        elif axis == 'Y':
            axis_indices = [4, 5, 10, 11, 16, 17]
        elif axis == 'Z':
            axis_indices = [0, 1, 6, 7, 12, 13]

        # Three series
        for series in range(3):
            down_idx = axis_indices[2 * series]
            up_idx   = axis_indices[2 * series + 1]

            w_down = omega_z_per_file[down_idx]
            w_up   = omega_z_per_file[up_idx]

            sin_phi_down = sin_phi_per_file[down_idx]
            sin_phi_up   = sin_phi_per_file[up_idx]
            # Use average sin(phi) for robustness
            sin_phi = (sin_phi_down + sin_phi_up) / 2.0

            if sin_phi < 1e-6:
                print(f"Warning: very small sin(phi)={sin_phi:.2e} for {axis} series {series+1}")
                sin_phi = 1e-6  # avoid division by zero

            # Formulas from your image
            b_g = (w_up + w_down) / 2.0
            S_g = (w_up - w_down - 2 * OMEGA_E * sin_phi) / (2 * OMEGA_E * sin_phi)

            results[axis].append({
                'series': series + 1,
                'file_down': csv_files[down_idx],
                'file_up':   csv_files[up_idx],
                'w_up': w_up,
                'w_down': w_down,
                'sin_phi': sin_phi,
                'b_g': b_g,
                'S_g': S_g
            })

    # ------------------------------------------------------------
    # 5. Print results
    # ------------------------------------------------------------
    def print_gyro_axis(axis):
        print(f"\n=== Gyro {axis}-axis (Earth rotation method) ===")
        print(f"{'Series':>6}  {'w_up':>12}  {'w_down':>12}  {'sin(phi)':>10}  {'b_g':>14}  {'S_g':>12}")
        print("-" * 68)
        for r in results[axis]:
            print(f"{r['series']:6d}  {r['w_up']:12.6e}  {r['w_down']:12.6e}  "
                  f"{r['sin_phi']:10.6f}  {r['b_g']:14.6e}  {r['S_g']:12.6f}")
        b_vals = [r['b_g'] for r in results[axis]]
        S_vals = [r['S_g'] for r in results[axis]]
        print("-" * 68)
        print(f"{'AVG':>6}  {'':>24}  {'':>10}  {np.mean(b_vals):14.6e}  {np.mean(S_vals):12.6f}")

    for ax in ['Z', 'X', 'Y']:
        print_gyro_axis(ax)

    # ------------------------------------------------------------
    # 6. Return structured data
    # ------------------------------------------------------------
    overall = {
        axis: {
            'b_g_avg': np.mean([r['b_g'] for r in results[axis]]),
            'S_g_avg': np.mean([r['S_g'] for r in results[axis]])
        } for axis in results
    }

    return {
        'per_series': results,
        'overall': overall,
        'sin_phi_per_file': sin_phi_per_file,
        'omega_e': OMEGA_E
    }


# =============================================================================
# Example usage:
# =============================================================================
if __name__ == "__main__":
    calib = calibrate_gyro_earth_rotation(
        '/content/drive/MyDrive/Twente/Q2/RPCN---November-2025/Assignment 1/Data/csv'
    )
    print("\nFinal averaged gyro calibration:")
    for axis in ['X', 'Y', 'Z']:
        print(f"{axis}-axis → b_g = {calib['overall'][axis]['b_g_avg']:+.6e} rad/s, "
              f"S_g = {calib['overall'][axis]['S_g_avg']:+.6f}")


=== Gyro Z-axis (Earth rotation method) ===
Series          w_up        w_down    sin(phi)             b_g           S_g
--------------------------------------------------------------------
     1  -2.251855e-03  -2.212495e-03   79.128569   -2.232175e-03     -1.003411
     2  -2.335373e-03  -2.249961e-03   80.751479   -2.292667e-03     -1.007252
     3  -2.413614e-03  -2.447524e-03   78.986773   -2.430569e-03     -0.997056
--------------------------------------------------------------------
   AVG                                         -2.318470e-03     -1.002573

=== Gyro X-axis (Earth rotation method) ===
Series          w_up        w_down    sin(phi)             b_g           S_g
--------------------------------------------------------------------
     1  -1.217030e-04  -3.968083e-05   85.552235   -8.069193e-05     -1.006574
     2  -2.172669e-04  -2.854233e-04   86.923380   -2.513451e-04     -0.994624
     3  -5.603399e-05  -1.877609e-04   86.770962   -1.218974e-04     -0.989591
